 # Bengaluru-House-Price-Prediction
The Prices are in lakhs (100k=1 lakhs).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
df=pd.read_csv('../input/bengaluru-house-price-data/Bengaluru_House_Data.csv')

In [ ]:
print(df.shape)

In [ ]:
df.head()

In [ ]:
df.info()

Data types:
* Categorical:area_type,availabilty,location,size,society
* continous: total_sqft,bath,balcony,price(to predict)

#### Removing columns that will be not used in making model

In [ ]:

df= df.drop(['area_type','society','balcony','availability'],axis='columns')
df.shape

# Removing Null values

In [ ]:
df.isnull().sum()

In [ ]:
df=df.dropna()

### **Features with missing values :location,size,society(a lot),bath,balcony**

 # Data cleaning
 

## total_sqft
* doesn't have null value


In [ ]:
df['total_sqft'].unique()

* Finding vlaues that can be and can't be converted to float.

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
df[~df['total_sqft'].apply(is_float)]

* There are some values in range for eg.(4323-4344) , converting them to float

In [ ]:
def convert_float(x):
    s=x.split('-')
    if len(s)==2:
        return (float(s[0
                       ])+float(s[1]))/2
    try:
        return float(x)
    except:
        return x    

In [ ]:
df['total_sqft']=df['total_sqft'].apply(convert_float)

* Again finding and cconveting the values that are not float.For eg. (1223Sq. Meters , 2432Sq. yards etc.)

In [ ]:
def unique_sqft(x):
    if(type(x)!=float):
        return True
    else:
        return False
        

In [ ]:
df[df['total_sqft'].apply(unique_sqft)]['total_sqft']

In [ ]:
def measurement_convert(x):
        if(type(x)==float):
            return x
        else:
            try:
                s=x.split('Sq.')
                if s[1].strip()=='Meter':
                    return (float(s[0])* 10.76391)
                elif s[1].strip()=='Yards':
                    return (float(s[0])*9)
            except:
                return None

In [ ]:
df['total_sqft']=df['total_sqft'].apply(measurement_convert)

In [ ]:
df.loc[872]#So it worked , you can check the value  by converting the total_sqft at index 872

_Still there are some measurements that we have not converted , but they are insignificant so we will say goodye(**drop**) to them._ (They are 14 in count.)

In [ ]:
df['total_sqft'].isnull().sum()

In [ ]:
df=df.dropna(subset=['total_sqft'])

In [ ]:
df2=df.copy()

In [ ]:
print(df.shape,df2.shape)#dropped 14 values ....horray

## Size (cleaning size column)

In [ ]:
df2['size']

In [ ]:
def bhk_convert(x):  
    try:
        b=x.split(" ")
        return int(b[0])
    except:
        return None    

In [ ]:
df2.dropna(subset=['size'],inplace=True)

In [ ]:
df2['bhk']=df2['size'].apply(bhk_convert)

In [ ]:
df2.head()

In [ ]:
df2.shape

In [ ]:
df2.drop('size',axis=1,inplace=True)

In [ ]:
df3=df2.copy()

In [ ]:
df3.head()

# Feature Engeneering 
Creating new feature price per_sqft, as it is significant for a property.

In [ ]:
df4=df3.copy()

In [ ]:
df4['price_per_sq_foot'] = (df4['price']*100000/df4['total_sqft'])

In [ ]:
df4.head()

In [ ]:
df5=df4.copy()

## Location column

In [ ]:
len(df5.location.unique())

There are a lot of different locations, we found no of data points for each location and consinder only those location having data points grater than _n_ data points (here say 10) and rename the location less than _n_ as 'others'.

In [ ]:
df5.location =df5.location.apply(lambda x : str(x).strip())
location_stats = df5.groupby('location')['location'].agg('count').sort_values(ascending=True)
location_stats.head()

In [ ]:
print("location with less than 10 data points: {}".format(len(location_stats[location_stats<=10])))
print("location with more than 10 data points: {}".format(len(location_stats[location_stats>10])))

Now , change all the location with data points less than or equal to 10 to 'other'

In [ ]:
location_lessthan_10=location_stats[location_stats <=10]

In [ ]:
df5.location=df5.location.apply(lambda x: "other" if x in location_lessthan_10 else x)

In [ ]:
df6=df5.copy()

## Outlier removal

## feature "bhk"

In [ ]:
df6.shape

In [ ]:
df6[df6.total_sqft/df6.bhk <300]

In [ ]:
df6 = df6[~(df6.total_sqft/df6.bhk<300)]

In [ ]:
df7 = df6.copy()

## feature "total_sqft"

In [ ]:
df7.price_per_sq_foot.describe()

In [ ]:
sns.distplot(df7['price_per_sq_foot'])

In [ ]:
def remove_pps_outlier(df):
    out_df = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sq_foot)
        st = np.std(subdf.price_per_sq_foot)
        reduced_df = subdf[(subdf.price_per_sq_foot>(m-st)) & (subdf.price_per_sq_foot<=(m+st))]
        out_df = pd.concat([out_df,reduced_df],ignore_index=True)
    return out_df        

In [ ]:
df7.shape

In [ ]:
df7 = remove_pps_outlier(df7)

In [ ]:
df7.shape

Visulaize the price comprasion between 2 Bhk and 3 bhk homes

In [ ]:
def visualize_bhk(df,location):
    bhk2=df[(df['bhk'] == 2) & (df['location'] == location)]
    bhk3=df[(df['bhk'] == 3 )& (df['location'] == location)]
    plt.figure(figsize=(15,7))
    plt.scatter(x=bhk2.total_sqft,y=bhk2.price,marker="o",label="bhk_2")
    plt.scatter(x=bhk3.total_sqft,y=bhk3.price,marker="+",label="bhk_3")
    plt.xlabel("Total_Sqft")
    plt.ylabel("price")
    plt.legend()

In [ ]:
visualize_bhk(df7,'JP Nagar')

We will remove bhk outlier  in differrent locations

In [ ]:
def remove_bhk_outlier(df):
    remove_indices = np.array([])
    for locatiion, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean':np.mean(bhk_df.price_per_sq_foot),
                'counts':bhk_df.shape[0],
                'std':np.std(bhk_df.price_per_sq_foot)
            }
            
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['counts']>5:
                remove_indices=np.append(remove_indices, bhk_df[bhk_df.price_per_sq_foot<stats['mean']].index.values)
    return df.drop(remove_indices, axis='index')
df8 = remove_bhk_outlier(df7)

In [ ]:
df8.shape #Number of rows reduced to 7353 


### Visulaize after removing the outliers
* One can observe that some of ouliters are removed and the data is now more well fitting to a line

In [ ]:
visualize_bhk(df8, 'JP Nagar')
visualize_bhk(df7, 'JP Nagar')

**We can see that the data is more noralized after removing bhk outliers**

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,7))
ax1.hist(df7['price_per_sq_foot'],edgecolor = 'black')
ax2.hist(df8['price_per_sq_foot'],color = 'green',edgecolor='black')
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")

# Bathroom outliers

In [ ]:
df8.bath.unique()

In [ ]:
df8.isnull().sum() # bath have some null values we will drop them

In [ ]:
sns.countplot(x='bath',data=df8)

In [ ]:
df8[df8.bath >10]

Remove all house with no. of bathrooms greater than bhk+2

In [ ]:
df9 = df8[~(df8.bath>df8.bhk+2)]

In [ ]:
df9.shape

# One-hot encoding for location

In [ ]:
dummies = pd.get_dummies(df9.location)

In [ ]:
df10=pd.concat([df9,dummies.drop('other',axis='columns')],axis='columns')

### Dropping columns location, price_per_sqft 

In [ ]:
df11 = df10.drop(['location', 'price_per_sq_foot'],axis='columns')

In [ ]:
df11.shape

# Building Model 

In [ ]:
X = df11.drop('price',axis='columns')
y = df11.price

## Spliting data into test and train datasets using test_train_split and  let's try liner regressor to train model

In [ ]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(X, y, test_size=.20, random_state=1)

In [ ]:
from sklearn.linear_model import LinearRegression

lg_model = LinearRegression()
lg_model.fit(train_x, train_y)
lg_model.score(val_x, val_y)

# Using k-fold cross validation to measure accuracy of algortihm 

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv=ShuffleSplit(n_splits=5, test_size=.2, random_state=2)
scores=cross_val_score(LinearRegression(), X, y, cv=cv)
scores

# Using Grid search CV

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X, y):
    algos = {
        'linear_regression':{
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        
        'lasso':{
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection':['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion':['mse', 'friedman_mse' ],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=1)
    for algo_name, settings in algos.items():
        gc = GridSearchCV(settings['model'], settings['params'], cv=cv, return_train_score=False)
        gc.fit(X,y)
        scores.append({
            "algo": algo_name,
            "best_score": gc.best_score_,
            "best_param": gc.best_params_,
            
        })
    return pd.DataFrame(scores, columns=["algo", "best_score", "best_param"])
        

In [ ]:
find_best_model_using_gridsearchcv(X, y)

## We observe that linear_regression performs best
Writing a function to predict price of house given the details


In [ ]:
def predict_price(location, total_sqft, bath, bhk ):
    loc_index = np.where(X.columns == location)[0][0]
    x = np.zeros(len(X.columns))
    x[0] = total_sqft
    x[1] = bath
    x[2] = bhk
    x[loc_index] = 1

    return lg_model.predict([x])[0]
    

In [ ]:
np.where(X.columns == "Sarjapur")

In [ ]:
predict_price("Electronic City Phase II",1056, 2 , 2)

In [ ]:
predict_price("Raja Rajeshwari Nagar", 1206, 2, 2 )

In [ ]:
import pickle 
with open("Blr_house_linear_regression.pickle", "wb") as f:
    pickle.dump(lg_model, f)

In [ ]:
import json 

columns = {
    'data_columns': [col.lower() for col in X.columns]
}

with open("data_columns.json","w") as j:
    j.write(json.dumps(columns))